<a href="https://colab.research.google.com/github/pseudope/2021_F_Level2/blob/main/Sessions/20211103_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 트랜스포머(Transformer)  
RNN을 사용하지 않고 인코더-디코더를 따르면서 어텐션 만으로 구현한 모델  
RNN만큼 성능이 우수함

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

#1. 기존 seq2seq 모델의 한계
인코더: 입력 시퀀스를 하나의 벡터 표현으로 압축  
디코더: 벡터 표현을 통해 출력 시퀀스를 만듦  
인코딩 과정에서 입력 시퀀스의 정보가 일부 손실됨 -> 어텐션 사용  
여기에서 어텐션을 RNN의 보정을 위한 용도가 아니라 어텐션으로 인코더와 디코더를 만듦

#2. 트랜스포머의 주요 하이퍼파라미터
dmodel=512  : 트랜스포머의 인코더와 디코더에서의 정해진 입력과 출력의 크기를 의미, 임베딩 벡터의 차원  
num_layers=6 : 트랜스포머 모델에서 인코더와 디코더가 총 몇 층으로 구성되었는지 의미  
num_heads=8 : 어텐션을 분할해서 병렬로 수행하는 병렬의 개수  
dff=2048 : 트랜스포머 내부에 있는 피드 포워드 신경망의 은닉층 크기

#3. 트랜스포머(Transformer)
인코더와 디코더가 N개로 구성  
<img src="https://wikidocs.net/images/page/31379/transformer2.PNG">
<img src="https://wikidocs.net/images/page/31379/transformer4_final_final_final.PNG">  
디코더는 시작 심볼 sos를 받아 종료 심볼 eos가 나올 때까지 연산 진행

#4. 포지셔널 인코딩(positional encoding)
트랜스포머의 입력: 단어 입력을 순차적으로 받지 않음 -> 포지셔널 인코딩  
포지셔널 인코딩: 단어의 위치 정보를 얻기 위해 각 단어의 임베딩 벡터에 위치 정보들을 더하여 모델의 입력으로 사용  
<img src="https://wikidocs.net/images/page/31379/transformer6_final.PNG">  
인코딩 덧셈은 임베딩 벡터가 모여 만들어진 문장 벡터 행렬과 포지셔널 인코딩 행렬 덧셈 연산을 통해 이루어진다.  
pos(입력 문장에서의 임베딩 벡터의 위치), i(임베딩 벡터 내의 차원의 인덱스)  
<img src="https://wikidocs.net/images/page/31379/transformer7.PNG">  
(pos, 2i) -> PE(pos, 2i)=sin(pos/10000^(2i/dmodel))  
(pos, 2i+1) -> PE(pos, 2i+1)=cos(pos/10000^(2i/dmodel))  

#5. 어텐션(Attention)
<img src="https://wikidocs.net/images/page/31379/attention.PNG">  
첫 번째: 인코더에서 이루어짐  
두 번째 & 세 번째: 디코더에서 이루어짐  
셀프 어텐션: Query, Key, Value가 동일한 경우  
세 번째는 Query는 디코더의 벡터이지만 Key와 Value가 인코더의 벡터이므로 셀프 어텐션이라 부르지 않는다.  
인코더의 셀프 어텐션 : Query = Key = Value  
디코더의 마스크드 셀프 어텐션 : Query = Key = Value  
디코더의 인코더-디코더 어텐션 : Query : 디코더 벡터 / Key = Value : 인코더 벡터  
<img src="https://wikidocs.net/images/page/31379/transformer_attention_overview.PNG">

#6. 인코더(Encoder)
<img src="https://wikidocs.net/images/page/31379/transformer9_final_ver.PNG">  
셀프 어텐션 & 피드 포워드 신경망  
멀티 헤드 셀프 어텐션: 셀프 어텐션을 병렬적으로 사용했다는 의미  

#7. 인코더의 셀프 어텐션
### 1)셀프 어텐션
Attention: Query에 대해서 모든 Key와의 유사도를 구하고 이 유사도를 가중치로 하여 키와 매핑되어있는 각각의 Value에 반영한다. 그리고 Value를 모두 가중합하여 리턴한다.  
<img src="https://wikidocs.net/images/page/22893/%EC%BF%BC%EB%A6%AC.PNG">  
Q = Querys : 모든 시점의 디코더 셀에서의 은닉 상태들  
K = Keys : 모든 시점의 인코더 셀의 은닉 상태들  
V = Values : 모든 시점의 인코더 셀의 은닉 상태들  
  
Self-Attention: 어텐션을 자기 자신에게 수행한다.  
Q : 입력 문장의 모든 단어 벡터들  
K : 입력 문장의 모든 단어 벡터들  
V : 입력 문장의 모든 단어 벡터들  
입력 문장 내의 단어들끼리 유사도를 구해 연관성을 찾는다.
###2) Q, K, V 벡터 얻기
각 단어 벡터들로부터 Q, K, V 벡터를 얻어 셀프 어텐션을 수행한다.  
dmodel(512)/num_heads(8)=64를 Q, K, V벡터의 차원으로 결정한다.  
student를 Q, K, V 벡터로 변환하는 과정  
<img src="https://wikidocs.net/images/page/31379/transformer11.PNG">
각 가중치 행렬: dmodel*(dmodel/num_heads) 크기

###3)스케일드 닷 프로덕트 어텐션(Scaled dot-product Attention)
Q 벡터는 모든 K벡터에 대해서 어텐션 스코어를 구하고 어텐션 분포를 구한 뒤에 이를 사용해 모든 V벡터를 가중합하여 어텐션 값 또는 컨텍스트 벡터를 구한다.  이를 모든 Q 벡터에 대해서 반복한다.  
<img src="https://wikidocs.net/images/page/31379/transformer13.PNG">  
주어진 score 함수를 사용하는 어텐션을 스케일드 닷-프로덕트 어텐션 이라고 한다.  
Attention Score: 각각 단어 I가 I, am, a, student와 얼마나 연관되어 있는지를 보여주는 값  
dk: 두 벡터의 내적값을 스케일링하는 값, K 벡터의 차원  
<img src="https://wikidocs.net/images/page/31379/transformer14_final.PNG">  
어텐션 스코어에 소프트맥스 함수를 사용해 어텐션 분포를 구하고 각 V 벡터와 가중합해 어텐션 값을 구한다.  
이 결과를 I에 대한 어텐션 값 또는 단어 I에 대한 컨텍스트 벡터라고 부른다.  

###4) 행렬 연산으로 일괄 처리하기
각 단어마다 벡터 연산을 하지 않고 행렬 연산으로 일괄 계산한다.  
<img src="https://wikidocs.net/images/page/31379/transformer12.PNG">  
<img src="https://wikidocs.net/images/page/31379/transformer15.PNG">  
<img src="https://wikidocs.net/images/page/31379/transformer16.PNG">  
각 단어의 어텐션 값을 모두 가지는 어텐션 값 행렬이 나온다.  

###6) 멀티 헤드 어텐션(Multi-head attention)
<img src="https://wikidocs.net/images/page/31379/transformer17.PNG">  
여러 번(num_heads)의 어텐션을 병렬로 사용하는 것이 더 효과적  
어텐션을 병렬로 수행해 다른 시각으로 정보들을 수집한다.  
어텐션 헤드: 각각의 어텐션 값 행렬, 가중치 행렬은 어텐션 헤드마다 전부 다름  
병렬 어텐션 수행 후 모든 어텐션 헤드를 연결(concatenate)  
<img src="https://wikidocs.net/images/page/31379/transformer18_final.PNG">  
어텐션 헤드를 모두 연결한 행렬은 또 다른 가중치 행렬을 곱하게 된다.  
<img src="https://wikidocs.net/images/page/31379/transformer19.PNG">  
Multi-head attention matrix=(seq_len, dmodle)크기  
첫 번째 서브층인 멀티-헤드 어텐션과 두 번째 서브층인 포지션 와이즈 피드 포워드 신경망을 지나며넛 인코더의 입력으로 들어올 때의 행렬의 크기가 계속 유지된다.  
따라서 인코더의 입력 크기가 출력에서도 동일 크기로 유지되어 다음 인코더에서도 다시 입력될 수 있다.  

###7) 패딩 마스크(padding mask)
입력 문장에 PAD 토큰이 있을 경우 어텐션에서 제외해 유사도를 구하지 않도록 한다.  
<img src="https://wikidocs.net/images/page/31379/pad_masking11.PNG">  
행: Query, 열: Key  
Key에 PAD가 있을 때 해당 열 전체를 마스킹(매우 작은 음수값을 넣음)한다.  
<img src="https://wikidocs.net/images/page/31379/pad_masking2.PNG">
<img src="https://wikidocs.net/images/page/31379/softmax.PNG">  
소프트맥스 함수 지난 후 0이 되어 유의미한 값을 지니지 않는다.  

#8. 포지션 와이즈 피드 포워드 신경망(Position-wise FFNN)
포지션 와이즈 FFNN: 완전 연결 FFNN(Fully-connected FFNN)  
FFNN(x)=MAX(0, xW1+b1)W2+b2  
<img src="https://wikidocs.net/images/page/31379/positionwiseffnn.PNG">  
x: 멀티 헤드 어텐션의 결과로 나온 행렬  
W1, b1, W2, b2: 한 인코더 층 내에서는 같은 값으로 사용, 인코더 층마다는 다른 값  

#9. 잔차 연결(Residual connection)과 층 정규화(Layer Normalization)
<img src="https://wikidocs.net/images/page/31379/transformer21.PNG">  
Add & Norm: 잔차 연결과 층 정규화
###1) 잔차 연결(Residual connection)
잔차 연결: 서브층의 입력과 출력을 더하는 것, H(x)=x+F(x)  
서브층의 입력과 출력이 같은 차원이므로 덧셈 연산이 가능  
서브층이 멀티 헤드 어텐션이 었을 때  
H(x)=x+Multi-head Attention(x)
<img src="https://wikidocs.net/images/page/31379/residual_connection.PNG">
###2) 층 정규화(Layer Normalization)
LN=LayerNorm(x+Sublayer(x))  
텐서의 마지막 차원(dmodel 차원)에 대해서 평균과 분산을 구하고, 이 값으로 어떤 수식을 통해 값을 정규화해 학습을 돕는다.  
<img src="https://wikidocs.net/images/page/31379/layer_norm_new_1_final.PNG">  
화살표 방향으로 각각 평균과 분산을 구함, xi는 각 화살표 방향의 벡터  
xi가 정규화 된 벡터 lni=LayerNorm(xi)  
1. 평균과 분산을 통해 정규화  
벡터 xi의 각 차원=k -> 벡터 xi의 각 k차원의 값 정규화(Xi,k)  
Xi,k=(xi,k - 평균)/sqrt(분산+e)   e(입실론):분모가 0이 되는 것을 막기 위한 값  
2. 감마와 베타를 도입  
<img src="https://wikidocs.net/images/page/31379/%EA%B0%90%EB%A7%88%EB%B2%A0%ED%83%80.PNG"> 초기값  
감마와 베타는 학습 가능한 파라마터  
최종 정규화 수식: lni=gamma*Xi+beta=LayerNorm(xi)

#12. 인코더에서 디코더로
<img src="https://wikidocs.net/images/page/31379/transformer_from_encoder_to_decoder.PNG">  
인코더 num_layers개의 층을 지나고 디코더로 넘어가 num_layers개의 디코더 층을 지난다. 디코더 연산마다 인코더가 보낸 출력을 각 디코더 층 연산에 사용한다.

#13. 디코더의 첫 번째 서브층: 셀프 어텐션과 룩-어헤드 마스크
디코더에도 임베딩 층과 포지셔널 인코딩을 거친 후의 문장 행렬이 입력된다.  
번역할 문장에 해당되는 문장 행렬을 한 번에 입력받고 각 시점의 단어를 예측하도록 훈련한다.  
트랜스포머는 문장 행렬로 입력을 한 번에 받아 현재 시점의 단어를 예측할 때 미래 시점의 단어까지도 참고할 수 있는 문제가 발생한다.  
첫 번째 서브층에서 미래의 단어를 참고하지 못하도록 룩-어헤드 마스크(look-ahead mask)사용  
<img src="https://wikidocs.net/images/page/31379/%EB%94%94%EC%BD%94%EB%8D%94.PNG">  
<img src="https://wikidocs.net/images/page/31379/decoder_attention_score_matrix.PNG">
<img src="https://wikidocs.net/images/page/31379/%EB%A3%A9%EC%96%B4%ED%97%A4%EB%93%9C%EB%A7%88%EC%8A%A4%ED%81%AC.PNG">  
  
트랜스포머 어텐션에서 함수에 전달하는 마스킹  
1. 인코더의 셀프 어텐션: 패딩 마스크 전달
2. 디코더의 첫 번째 서브층인 마스크드 셀프 어텐션: 룩-어헤드 마스크 전달
3. 디코더의 두 번째 서브층인 인코더-디코더 어텐션: 패딩 마스크 전달

룩-어헤드 마스크에서도 패딩 마스크가 필요하다.

#14. 디코더의 두 번째 서브층: 인코더-디코더 어텐션
멀티 헤드 어텐션이나 셀프 어텐션은 아니다.  
Query는 디코더 행렬, Key와 Value는 인코더 행렬  
Key, Value: 인코더의 마지막 층에서 온 행렬로부터 얻는다.  
Query: 디코더의 첫 번째 서브층의 결과 행렬로부터 얻는다.  
<img src="https://wikidocs.net/images/page/31379/%EB%94%94%EC%BD%94%EB%8D%94%EB%91%90%EB%B2%88%EC%A7%B8%EC%84%9C%EB%B8%8C%EC%B8%B5%EC%9D%98%EC%96%B4%ED%85%90%EC%85%98%EC%8A%A4%EC%BD%94%EC%96%B4%ED%96%89%EB%A0%AC_final.PNG">  